Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analysis

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [4]:
#FourSquare API Function  
def citybike_fetch(query, long, lat):             
    # Set Keys                          
    # Set Parameters for API         
    network_id = 'velo-antwerpen'
    bike_url = ("http://api.citybik.es/v2/networks/"+network_id)       
    location = f'{long},{lat}'         
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3EWjrDVS7MmpNxPS2oYo9CdBphNOQ/lOuCT1N7N4xCzo="}
    
    bike_response = requests.request("GET", bike_url, params={'ll':location,                                                   
                                                      'radius':1000,                                                                                                      
                                                      'query':query,                                                   
                                                      "sort":"DISTANCE"},
                                                  headers=headers) 
    
    res_json = bike_response.json()
    citybike_df = pd.json_normalize(res_json)
    return citybike_df

In [5]:
response = citybike_fetch(query="coffee", long="51.2162206", lat="4.421068612450207")

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
#Foursquare API

Look at the data before and after the join to validate your data.

In [6]:
# FourSquare API Function  
def foursquare_fetch(query, long, lat):             
    # Set Keys                          
    # Set Parameters for API         
    fs_url = "https://api.foursquare.com/v3/places/search"         
    location = f'{long},{lat}'         
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3EWjrDVS7MmpNxPS2oYo9CdBphNOQ/lOuCT1N7N4xCzo="}
    
    fs_response = requests.request("GET", fs_url, params={'ll':location,                                                   
                                                      'radius':1000,                                                                                                      
                                                      'query':query,                                                   
                                                      "sort":"DISTANCE"},
                                                  headers=headers) 
    
    res_json = fs_response.json()
    fs_df = pd.json_normalize(res_json)
    return fs_df

In [7]:
response = foursquare_fetch(query="coffee", long="51.2162206", lat="4.421068612450207")

In [8]:
coordinates = citybike_df[['longitude', "latitude"]] 
response_list = [] 
for every  in coordinates:     
    #grab longitude, latitude as variables     
    longitude = ['longtitude']     
    latitude = ['latitude']     
    response_list.append(foursquare_fetch(query="coffee", long=longitude, lat=latitude)) 
total_df_fs = pd.concat(response_list)

NameError: name 'citybike_df' is not defined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fsq_ylp_df = pd.merge(fs_df,yelp_df, how='outer') 
fsq_ylp_df

In [ ]:
fsq_ylp_df.describe()
sns.histplot(fsq_ylp_df['latitude'])

In [ ]:
cormat = fsq_ylp_df.corr()
top_corr_features = cormat.index[abs(cormat["latitude"])>0.5]
plt.figure(figsize=(10,10))
g = sns.heatmap(fsq_ylp_df[top_corr_features].corr(),annot=True,cmap="RdYlGn_r")

In [ ]:
var="latitude"
data=fsq_ylp_df[['latitude',var]]
data.plot.scatter(x=var, y='latitude', ylim=(0,800000))

In [ ]:
#DATABASE JOIN

In [ ]:
connection = create_connection("fsq_ylp.sqlite")

In [ ]:
# to define the function
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"the error '{e}' occured")
        

In [ ]:
# query for the table
create_rating_table = """
CREATE TABLE IF NOT EXISTS latitude(
        id VARCHAR(100) PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(25) NOT NULL,
        distance DECIMAL,
        rating DECIMAL);"""

In [ ]:
execute_query(connection, create_rating_table)

In [ ]:
create_popularity_table = """
CREATE TABLE IF NOT EXISTs popularity(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        distance DECIMAL,
        popularity float,
        FOREIGN KEY (distance) REFERENCES latitude(distance));"""

In [ ]:
execute_query(connection, create_popularity_table)